Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD        Naive SE       MCSE      ESS
alpha 154.59784175 0.271339329 0.00429025149 0.0033574491 1000
 beta   0.90428598 0.041472675 0.00065574056 0.0007201006 1000
sigma   5.10256046 0.191788492 0.00303244232 0.0028268468 1000

Quantiles:
          2.5%       25.0%       50.0%       75.0%       97.5%   
alpha 154.0699750 154.4100000 154.5965000 154.7790000 155.1240000
 beta   0.8208997   0.8762495   0.9044275   0.9325775   0.9844677
sigma   4.7447682   4.9692775   5.0955450   5.2305750   5.5008560



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.901 0.944651 5.12858; 154.247 0.881037 4.98284; … ; 154.801 0.965011 5.09543; 154.594 0.910901 4.99157]

Union{Missing, Float64}[154.444 0.91241 5.07742; 154.563 0.916036 4.8026; … ; 154.85 0.856148 4.93002; 154.863 0.860809 4.96007]

Union{Missing, Float64}[154.327 0.907818 5.07858; 154.584 0.933755 5.29625; … ; 154.082 0.829631 5.16853; 154.71 0.936974 4.97542]

Union{Missing, Float64}[153.992 0.937123 5.26909; 154.641 0.901063 4.76295; … ; 154.71 0.93222 4.98879; 154.558 0.872472 5.10844]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD        Naive SE       MCSE      ESS
alpha 154.59784175 0.271339329 0.00429025149 0.0033574491 1000
 beta   0.90428598 0.041472675 0.00065574056 0.0007201006 1000
sigma   5.10256046 0.191788492 0.00303244232 0.0028268468 1000

Quantiles:
          2.5%       25.0%       50.0%       75.0%       97.5%   
alpha 154.0699750 154.4100000 154.5965000 154.7790000 155.1240000
 beta   0.8208997   0.8762495   0.9044275   0.9325775   0.9844677
sigma   4.7447682   4.9692775   5.0955450   5.2305750   5.5008560



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*